In [1]:
import requests
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
import os
import re
from urllib.request import urlopen
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np

In [5]:
def hashtag_links(hashtag):

    '''Code to return url pages where a hashtag has been used'''
    
    #####CHANGE#### - This is the location of chromedriver on my computer
    chromedriver = "C:/webdrivers/chromedriver"
    browser = webdriver.Chrome(chromedriver)
    
    #Opens the instgram tags pages and searches the hashtag
    browser.get('https://www.instagram.com/explore/tags/'+hashtag)
    Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight/1.5);")
    links=[]
    source = browser.page_source
    data=bs(source, 'html.parser')
    body = data.find('body')
    script = body.find('span')
    for link in script.findAll('a'):
         if re.match("/p", link.get('href')):
             links.append('https://www.instagram.com'+link.get('href'))
            
    #sleep time is required. If you don't use this Instagram may interrupt the script and doesn't scroll through pages
    time.sleep(5) 
    Pagelength = browser.execute_script("window.scrollTo(document.body.scrollHeight/1.5, document.body.scrollHeight/3.0);")
    source = browser.page_source
    data=bs(source, 'html.parser')
    body = data.find('body')
    script = body.find('span')
    for link in script.findAll('a'):
         if re.match("/p", link.get('href')):
                links.append('https://www.instagram.com'+link.get('href'))

    # Create a zipped list of tuples from above lists
    zippedList =  list(zip(links))

    # Create a dataframe from zipped list
    dfObj = pd.DataFrame(zippedList, columns = ['Url_links']) 
    dfObj.to_csv('hashtag_links.csv')

In [11]:
# If you want to scrape the information from one user
def recent_posts(username, number):
    
    # number = how many posts you want a url for 
    
    """With the input of an account page, scrape the 'number' of most recent posts urls"""
        
    #####CHANGE#### - This is the location of chromedriver on my computer
    chromedriver = "C:/webdrivers/chromedriver"
    browser = webdriver.Chrome(chromedriver)
    
    #define url
    url = "https://www.instagram.com/" + username + "/"
    
    #use chrome to launch url
    browser.get(url)
    post = 'https://www.instagram.com/p/'
    
    post_links = []
    
    while len(post_links) < number:
        links = [a.get_attribute('href') for a in browser.find_elements_by_tag_name('a')]
        for link in links:
            if post in link and link not in post_links:
                post_links.append(link)
        scroll_down = "window.scrollTo(0, document.body.scrollHeight);"
        browser.execute_script(scroll_down)
        time.sleep(10)
    else:
        # Create a zipped list of tuples from above lists
        zippedList =  list(zip(post_links))

        # Create a dataframe from zipped list
        dfObj = pd.DataFrame(zippedList, columns = ['Url_links']) 
        
        
        return dfObj.to_csv('user_links.csv')

In [15]:
recent_posts('bobbytonylowe', 3)

In [6]:
def instagram_crawler(base_url):
    chromedriver = "C:/webdrivers/chromedriver"
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    actions = ActionChains(driver)

    driver.get(base_url)
    Pagelength = driver.execute_script("window.scrollTo(0, document.body.scrollHeight/1.5);")

    images = driver.find_elements_by_class_name("_bz0w")
    #image_curr = images[0].find_element_by_tag_name("a").get_attribute("href")
    #driver.get(image_curr) #go to first picture

    comments = driver.find_elements_by_class_name("C4VMK")

    comments_list, users_list = [], []

    for c in comments:
        comment = c.find_element_by_css_selector('span').get_attribute("textContent")
        user = c.find_element_by_class_name("TlrDj").get_attribute("textContent")
        
        #print("" + user + ": " + str(comment))
        comments_list.append(comment)
        users_list.append(user)
   

    #sleep time is required. If you don't use this Instagram may interrupt the script and doesn't scroll through pages
    time.sleep(5) 
        
    df = pd.DataFrame({"user": users_list, "comment": comments_list})
    return df


In [7]:
instagram_crawler("https://www.instagram.com/p/B08-mLEnNR8/")

,user,comment
0,motorsport.video,Indycar is a great series for racing fans🔥Is i...
1,israelmorrison,The machines are coming
2,pivotles,None of these guys could hang in F1
3,rubencitymadhouse14,"Just like an F1 race there’s no contest, Indyc..."
4,sahil_chandnani98,Best part is that there are a lot of cars on t...
5,raymarshjr,Most definitely....better competition in Indyc...
6,kevinscarz,In terms of pure racing Indycar is superior be...
7,kyyyyyhan,Nothing better than F1
8,franciscosta_4327,Y E S
9,wentidizi,No!


In [113]:
# function to create an excel file with multiple sheets
def dfs_tabs(df_list, sheet_list, file_name):
    
    '''function to create an excel file with multiple sheets '''
    
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   

    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
        
    writer.save()

In [18]:
def get_name(url):
    
    try:    
        html = urllib.request.urlopen(url)
        soup = BeautifulSoup(html, 'html.parser')
        data = soup.find_all('meta', attrs={'property': 'og:description'
                             })
        text = data[0].get('content').split()
        user = '%s %s %s' % (text[-3], text[-2], text[-1])
        followers = text[0]
        following = text[2]
        posts = text[4]
        return user[0:18]
    except:
        pass
        return 'unknown'
    


In [19]:
get_name('https://www.instagram.com/bobbytonylowe/')

'Bobby Lowe (@bobby'

In [8]:
def getprofile(url):
    html = urllib.request.urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    data = soup.find_all('meta', attrs={'property': 'og:description'
                         })
    text = data[0].get('content').split()
    user = '%s %s %s' % (text[-3], text[-2], text[-1])
    followers = text[0]
    following = text[2]
    posts = text[4]
    print ('User:', user)
    print ('Followers:', followers)
    print ('Following:', following)
    print ('Posts:', posts)
    print ('---------------------------')
getprofile("https://www.instagram.com/bobbytonylowe/")

User: Bobby Lowe (@bobbytonylowe)
Followers: 407
Following: 706
Posts: 192
---------------------------


In [122]:
# List the url links
df =  pd.read_csv('hashtag_links.csv')
df = df['hashtags']
page_list = list(str(e) for e in df)

In [ ]:
#Name of each instgram user for each of the instgram posts
sheet_name = [get_name(x) for x in page_list]

#List comprehension to create a list of dataframes created from the instagram crawler function 
dfs = [instagram_crawler(x) for x in page_list]

#Perform the multiple excel sheet function on the selected dataframes
dfs_tabs(dfs, sheet_name, 'instagram_comments.xlsx')